# ETL PROJECT Group 2 - Sri Vegunta and Josh Cullen

### Import Dependencies and Setup

In [97]:
import pandas as pd
import numpy as np 
import datetime as dt
from dateutil.relativedelta import relativedelta
from sqlalchemy import create_engine

### Create Aggregate Tables for the Season Data to identify Best Players in the Season

In [98]:
# Import Season Data
csv = "Resources/season.csv"
# Read Seasons Data and store into Pandas data frame
season_data_df = pd.read_csv( csv)
season_data_df.head()

,Season_Id,Season_Year,Orange_Cap_Id,Purple_Cap_Id,Man_of_the_Series_Id
0,1,2008,100,102,32
1,2,2009,18,61,53
2,3,2010,133,131,133
3,4,2011,162,194,162
4,5,2012,162,190,315


In [99]:
# Place all of the orangecap(Best Batsman) data into a summary DataFrame
orangecap_group = pd.DataFrame(season_data_df["Orange_Cap_Id"].value_counts())
#Reset the Index
orangecap_group.reset_index(inplace=True)
#Assign name for the column variables
orangecap_group.columns = ["Player_Id","Series_BestBatsman"]
orangecap_group

,Player_Id,Series_BestBatsman
0,162,2
1,46,1
2,187,1
3,8,1
4,133,1
5,100,1
6,19,1
7,18,1


In [100]:
# Place all of the purplecap(Best Bowler) data into a summary DataFrame
purplecap_group = pd.DataFrame(season_data_df["Purple_Cap_Id"].value_counts())
#Reset the Index
purplecap_group.reset_index(inplace=True)
#Assign name for the column variables
purplecap_group.columns = ["Player_Id","Series_BestBowler"]
purplecap_group

,Player_Id,Series_BestBowler
0,71,2
1,190,1
2,61,1
3,364,1
4,299,1
5,102,1
6,131,1
7,194,1


In [109]:
# Place all of the Man of the Series data into a summary DataFrame
manofseries_group = pd.DataFrame(season_data_df["Man_of_the_Series_Id"].value_counts())
#Reset the Index
manofseries_group.reset_index(inplace=True)
#Assign name for the column variables
manofseries_group.columns = ["Player_Id","Manoftheseries"]
manofseries_group

,Player_Id,Manoftheseries
0,32,2
1,133,1
2,315,1
3,8,1
4,334,1
5,53,1
6,162,1
7,305,1


# FINAL PLAYER TABLE DATA TRANSFORMATION

### Clean, Create new variables,Merge best player aggregate Data to the Player Table to make a new Player Table

In [118]:
# Import PLayer Data
csv_file = "Resources/Player.csv"
player_data_df = pd.read_csv(csv_file)

#Introduce a new varible to identify player's age 
# Coverting Date from a string format to a datetime format
player_data_df['DateofBirth'] = pd.to_datetime(player_data_df.DOB)
# This function converts given date to age
player_data_df["age"] = (pd.to_datetime('now') - player_data_df["DateofBirth"]).astype('<m8[Y]')

#Introduce Country Code Variable for the Player Table to identify countries by codes
player_data_df.loc[player_data_df["Country"] == 'Australia', 'Cntry_code'] = 'AUS'
player_data_df.loc[player_data_df["Country"] == 'Bangladesh', 'Cntry_code'] = 'BGD'  
player_data_df.loc[player_data_df["Country"] == 'England', 'Cntry_code'] = 'GBR'  
player_data_df.loc[player_data_df["Country"] == 'India', 'Cntry_code'] = 'IND'  
player_data_df.loc[player_data_df["Country"] == 'Netherlands', 'Cntry_code'] = 'NLD'  
player_data_df.loc[player_data_df["Country"] == 'New Zealand', 'Cntry_code'] = 'NZL'  
player_data_df.loc[player_data_df["Country"] == 'Pakistan', 'Cntry_code'] = 'PAK'  
player_data_df.loc[player_data_df["Country"] == 'South Africa', 'Cntry_code'] = 'ZAF'  
player_data_df.loc[player_data_df["Country"] == 'Sri Lanka', 'Cntry_code'] = 'LKA'  
player_data_df.loc[player_data_df["Country"] == 'West Indies', 'Cntry_code'] = 'WIN'  
player_data_df.loc[player_data_df["Country"] == 'Zimbabwea', 'Cntry_code'] = 'ZWE'  

# Merge orangecap_group to the Players Table to identify players who won the best Batsman in the series
player_data_df1 = pd.merge(player_data_df,orangecap_group,on = "Player_Id",how = 'left')

# Merge purplecap_group to the Players Table to identify players who won the best Bowler in the series
player_data_df2 = pd.merge(player_data_df1,purplecap_group,on = "Player_Id",how = 'left')

# Merge manofseries_group to the Players Table to identify players who won the Man of the series
player_data_df3 = pd.merge(player_data_df2,manofseries_group,on = "Player_Id",how = 'left')

#Sort the Top players by the players who won the most Manofthematch in the Series
topplayers = player_data_df3.sort_values(by=["Manoftheseries"], ascending = (False))

# Data Cleaning 

#Removed column that had data that was incorrect. 
del topplayers["Unnamed: 7"] 

#Removed int DateofBirth Varible and retain original DOB String Variable
del topplayers["DateofBirth"] 

#Replace all empty values with "N/A - Umpire" value for Batting_Hand and Bowling_Skill when the player 
#is an umplire
topplayers.loc[topplayers['Is_Umpire'] == 1, 'DOB'] = 'N/A - Umpire'
topplayers.loc[topplayers['Is_Umpire'] == 1, 'Batting_Hand'] = 'N/A - Umpire'
topplayers.loc[topplayers['Is_Umpire'] == 1, 'Bowling_Skill'] = 'N/A - Umpire'

#Replace all empty values with "No Bowling Skill" for the variable Bowling_Skill when players do not have a Bowling skill
topplayers['Bowling_Skill'] = topplayers['Bowling_Skill'].fillna('No Bowling Skill')

#Replace all empty values with 0 value for players that haven't won anything in the series
topplayers['Series_BestBatsman'] = topplayers['Series_BestBatsman'].fillna(0)
topplayers['Series_BestBowler'] = topplayers['Series_BestBowler'].fillna(0)
topplayers['Manoftheseries'] = topplayers['Manoftheseries'].fillna(0)
topplayers['age'] = topplayers['age'].fillna(0)

topplayers

,Player_Id,Player_Name,DOB,Batting_Hand,Bowling_Skill,Country,Is_Umpire,age,Cntry_code,Series_BestBatsman,Series_BestBowler,Manoftheseries
31,32,SR Watson,17-Jun-81,Right_Hand,Right-arm fast-medium,Australia,0,39.0,AUS,0.0,0.0,2.0
7,8,V Kohli,5-Nov-88,Right_Hand,Right-arm medium,India,0,32.0,IND,1.0,0.0,1.0
52,53,AC Gilchrist,14-Nov-71,Left_Hand,Right-arm offbreak,Australia,0,49.0,AUS,0.0,0.0,1.0
132,133,SR Tendulkar,24-Apr-73,Right_Hand,Right-arm offbreak,India,0,48.0,IND,1.0,0.0,1.0
161,162,CH Gayle,21-Sep-79,Left_Hand,Right-arm offbreak,West Indies,0,41.0,WIN,2.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...
518,519,Subroto Das,N/A - Umpire,N/A - Umpire,N/A - Umpire,India,1,0.0,IND,0.0,0.0,0.0
519,520,K Srinivasan,N/A - Umpire,N/A - Umpire,N/A - Umpire,India,1,0.0,IND,0.0,0.0,0.0
520,521,VK Sharma,N/A - Umpire,N/A - Umpire,N/A - Umpire,India,1,0.0,IND,0.0,0.0,0.0
521,523,AV Wankhade,14-Mar-92,Right_Hand,No Bowling Skill,India,0,29.0,IND,0.0,0.0,0.0


# FINAL MATCH TABLE DATA TRANSFORMATION

### Merge Team and Season Data to Match to have all information in Match Data table 
### Eliminate the need for Team and Season Data whilst retaining unique match ID's

In [135]:
# Import Original Match Data
csv_match = "Resources/Match.csv"
# Read Purchasing File and store into Pandas data frame
match_data_df = pd.read_csv(csv_match)

# Import Original Team Data
csv_team = "Resources/Team.csv"
# Read Purchasing File and store into Pandas data frame
team_data_df = pd.read_csv(csv_team)

# Import Original Player Match Data
csv_player_match = "Resources/Player_Match.csv"
# Read Purchasing File and store into Pandas data frame
player_match_df = pd.read_csv(csv_player_match)

# Merging Team info to Match Data to remove the need for team info table - First Team Details Acquired
match_data_df1 = pd.merge(match_data_df, team_data_df, left_on='Team_Name_Id', right_on='Team_Id')
match_data_df2 = match_data_df1.rename(columns={"Team_Name": "Team1_Name", "Team_Short_Code": "Team1_Short_Code"})
match_data_df3 = match_data_df2.drop(['Team_Id'], axis = 1)

# Merging Team info to Match Data to remove the need for team info table - Opponent Team Details Acquired
match_data_df4 = pd.merge(match_data_df3, team_data_df, left_on='Opponent_Team_Id', right_on='Team_Id')
match_data_df5 = match_data_df4.rename(columns={"Team_Name": "Opponent_Team_Name", "Team_Short_Code": "OpponentTeam_Short_Code"})

# Data Cleaning 

#Remove Team_Id column 
del match_data_df5["Team_Id"] 

#Fill all empty values with 0 value when Result is a draw
match_data_df5.loc[match_data_df5['IS_Result'] == 0, 'Won_By'] = '0'
match_data_df5.loc[match_data_df5['IS_Superover'] == 1, 'Won_By'] = '0'
match_data_df5.loc[match_data_df5['IS_Result'] == 0, 'Match_Winner_Id'] = '0'
match_data_df5.loc[match_data_df5['IS_Result'] == 0, 'Man_Of_The_Match_Id'] = '0'

# Merging new Match Data with team info with Season data to eliminate the need for Season table
Match_Team_Season = pd.merge(match_data_df5, season_data_df, left_on='Season_Id', right_on='Season_Id')
Match_Team_Season

,Match_Id,Match_Date,Team_Name_Id,Opponent_Team_Id,Season_Id,Venue_Name,Toss_Winner_Id,Toss_Decision,IS_Superover,IS_Result,...,City_Name,Host_Country,Team1_Name,Team1_Short_Code,Opponent_Team_Name,OpponentTeam_Short_Code,Season_Year,Orange_Cap_Id,Purple_Cap_Id,Man_of_the_Series_Id
0,335987,18-Apr-08,2,1,1,M Chinnaswamy Stadium,2,field,0,1,...,Bangalore,India,Royal Challengers Bangalore,RCB,Kolkata Knight Riders,KKR,2008,100,102,32
1,336008,3-May-08,4,1,1,"Punjab Cricket Association Stadium, Mohali",4,bat,0,1,...,Chandigarh,India,Kings XI Punjab,KXIP,Kolkata Knight Riders,KKR,2008,100,102,32
2,336026,16-May-08,7,1,1,Wankhede Stadium,7,field,0,1,...,Mumbai,India,Mumbai Indians,MI,Kolkata Knight Riders,KKR,2008,100,102,32
3,336005,1-May-08,5,1,1,Sawai Mansingh Stadium,5,bat,0,1,...,Jaipur,India,Rajasthan Royals,RR,Kolkata Knight Riders,KKR,2008,100,102,32
4,336019,11-May-08,8,1,1,"Rajiv Gandhi International Stadium, Uppal",1,bat,0,1,...,Hyderabad,India,Deccan Chargers,DC,Kolkata Knight Riders,KKR,2008,100,102,32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
572,981000,16-May-16,1,2,9,Eden Gardens,2,field,0,1,...,Kolkata,India,Kolkata Knight Riders,KKR,Royal Challengers Bangalore,RCB,2016,8,299,8
573,980958,30-Apr-16,11,2,9,"Rajiv Gandhi International Stadium, Uppal",2,field,0,1,...,Hyderabad,India,Sunrisers Hyderabad,SRH,Royal Challengers Bangalore,RCB,2016,8,299,8
574,980942,24-Apr-16,13,2,9,Saurashtra Cricket Association Stadium,2,bat,0,1,...,Rajkot,India,Gujarat Lions,GL,Royal Challengers Bangalore,RCB,2016,8,299,8
575,981018,24-May-16,13,2,9,M Chinnaswamy Stadium,2,field,0,1,...,Bangalore,India,Gujarat Lions,GL,Royal Challengers Bangalore,RCB,2016,8,299,8


# EXPORT FILES TO BE IMPORTED INTO SQL

In [136]:
# Export New Player Data
csv = "FINALIPL_Data/Player.csv"
# Export Players Dataframe to a csv file to upload into SQL
topplayers.to_csv(csv,index = False)

# Export New Match Data
csv = "FINALIPL_Data/Match_Team_Season.csv"
# Export Players Dataframe to a csv file to upload into SQL
Match_Team_Season.to_csv(csv,index = False)

# Export Player_Match Data
csv = "FINALIPL_Data/Player_Match.csv"
# Export Players Dataframe to a csv file to upload into SQL
player_match_df.to_csv(csv,index = False)